In [1]:
input = {
    "pH(산도)" :	6.0,
"유기물(OM)" :	24.0,
"유효인산(P2O5)" :	1333.0,
"칼륨(K)" :	0.500,
"칼슘(Ca)"	: 0.0,
"마그네슘(Mg)" :	0.000,
"전기전도도(EC)" :0.000}

In [ ]:
import xml.etree.ElementTree as ET
import xmltodict
import json
import requests

serviceKey="q+kAKJCJgJXlNlBFxk5LHCmDivqtHEVdmd3vh4cftkCafbEmv4agKxoZemYjbqE9Gxjy0lRCVmbcG3ZtR4K2Tw=="
url = 'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExam'
params ={'serviceKey' : serviceKey, 'PNU_Code' : '5115034022100750001' }

response = requests.get(url, params=params)
response_context = response.text
#### parsing to json
response_json = xmltodict.parse(response_context)["response"]
input_data = response_json["body"]["items"]["item"]
input_data

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# 🔹 OpenAI 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

# 🔹 참고 문서 (작물별 적정 환경)
context = [
    {
        "작물": "기장",
        "pH(산도)": (6.5, 7.0),
        "유기물(OM)": (20, 30),
        "유효인산(P2O5)": (150, 250),
        "칼륨(K)": (0.45, 0.55),
        "칼슘(Ca)": (6.0, 7.0),
        "마그네슘(Mg)": (2.0, 2.5),
        "전기전도도(EC)": (10, 15)
    },
    {
        "작물": "강활",
        "pH(산도)": (6.5, 7.0),
        "유기물(OM)": (25, 30),
        "유효인산(P2O5)": (150, 250),
        "칼륨(K)": (0.45, 0.55),
        "칼슘(Ca)": (5.0, 6.0),
        "마그네슘(Mg)": (1.5, 2.0),
        "전기전도도(EC)": (10, 15)
    }
]

# 🔹 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    아래의 토양 환경 정보를 참고하여 키울 수 있는 가장 적합한 작물을 추천해 주세요.
    
    🌱 **사용자 입력 (토양 정보)**:
    {input_data}
    
    📄 **참고 문서 (작물별 적정 환경)**:
    {context}
    
    비교 후 가장 적합한 작물을 추천하고, 그 이유도 설명해 주세요.
    """,
    input_variables=["input_data", "context"]
)

# 🔹 체인 생성 및 실행
chain = prompt | model
response = chain.invoke({"input_data": input_data, "context": context})

# 🔹 결과 출력
response.pretty_print()


C:\Users\shinc\AppData\Local\Temp\ipykernel_1036\3791885048.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o-mini")


================================== Ai Message ==================================

제공된 토양 환경 정보를 바탕으로 추천할 수 있는 작물은 **기장**입니다.

### 토양 정보:
- pH: 6.800
- 유기물(OM): 13.000
- 칼륨(K): 0.174
- 칼슘(Ca): 3.500

### 작물별 적정 환경:
1. **기장**
   - pH(산도): (6.5, 7.0)
   - 유기물(OM): (20, 30)
   - 칼륨(K): (0.45, 0.55)
   - 칼슘(Ca): (6.0, 7.0)

2. **강활**
   - pH(산도): (6.5, 7.0)
   - 유기물(OM): (25, 30)
   - 칼륨(K): (0.45, 0.55)
   - 칼슘(Ca): (5.0, 6.0)

### 비교 및 추천 이유:
- **pH**: 두 작물 모두 pH 범위가 적합합니다. 현재 토양 pH는 6.800으로 두 작물의 적정 범위(6.5 ~ 7.0) 내에 있습니다.
- **유기물(OM)**: 기장의 적정 유기물 범위는 20~30%이고, 현재 유기물 함량은 13%로 낮습니다. 강활의 적정 유기물 함량도 비슷하지만, 두 작물 모두 현재 유기물 함량이 부족합니다.
- **칼륨(K)**: 현재 칼륨 함량은 0.174로 두 작물의 적정 범위(0.45 ~ 0.55)보다 낮습니다. 이는 두 작물 모두에게 적합한 환경이 아닙니다.
- **칼슘(Ca)**: 현재 칼슘 함량도 3.5로 두 작물의 적정 범위(기장: 6.0~7.0, 강활: 5.0~6.0)보다 낮습니다.

### 결론:
두 작물 모두 현재 토양의 유기물, 칼륨, 칼슘 함량이 부족하지만, pH 조건이 두 작물 모두 적합합니다. 그러나 기장이 강활에 비해 더 적합할 수 있는 이유는 기장이 일반적으로 더 널리 재배되고 다양한 환경에서 자생성을 가지는 작물이기 때문입니다. 따라서 기장을 추천드립니다. 

추가적인 비료나 토양 개량제를 사용하여 유기물과 칼륨, 칼

# csv 파일로 참고 자료 제공. 

In [1]:
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [ ]:
# 🔹 참고 문서 (작물별 적정 환경)
sample = pd.read_csv("흙토람 최종.csv")
context = sample
sample

,작물,산도,전기전도도,질산태질소,유기물,유효인산,칼륨,칼슘,마그네슘,양이온교환용량,붕소
0,가지,6.0~6.5,≤2,70~200,20~30,400~500,0.70~0.80,5.0~6.0,1.5~2.0,10~15,NaN
1,감나무,6.0~6.5,NaN,NaN,25~35,200~300,0.30~0.60,5.0~6.0,1.5~2.0,10~15,NaN
2,감자,5.5~6.2,≤2,NaN,20~30,250~350,0.50~0.60,4.5~5.5,1.5~2.0,10~15,NaN
3,감초,6.0~6.5,NaN,NaN,25~30,150~250,0.45~0.55,5.0~6.0,1.5~2.0,10~15,NaN
4,강낭콩,6.5~7.0,NaN,NaN,20~30,150~250,0.45~0.55,6.0~7.0,2.0~2.5,10~15,NaN
...,...,...,...,...,...,...,...,...,...,...,...
155,피망,6.0~6.5,≤2,70~200,25~35,450~550,0.70~0.80,5.0~6.0,1.5~2.0,10~15,NaN
156,호박,6.0~6.5,≤2,70~200,20~30,350~450,0.70~0.80,5.0~6.0,1.5~2.0,10~15,NaN
157,황금,6.0~6.5,NaN,NaN,25~30,150~250,0.45~0.55,5.0~6.0,1.5~2.0,10~15,NaN
158,황기,6.0~6.5,NaN,NaN,25~30,150~250,0.45~0.55,5.0~6.0,1.5~2.0,10~15,NaN


In [ ]:
# 🔹 OpenAI 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

# 🔹 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    아래의 토양 환경 정보를 참고하여 키울 수 있는 가장 적합한 작물을 추천해 주세요.
    
    🌱 **사용자 입력 (토양 정보)**:
    {input_data}
    
    📄 **참고 문서 (작물별 적정 환경)**:
    {context}
    
    비교 후 가장 적합한 작물을 추천하고, 그 이유도 설명해 주세요.
    """,
    input_variables=["input_data", "context"]
)

# 🔹 체인 생성 및 실행
chain = prompt | model
response = chain.invoke({"input_data": input_data, "context": context})

# 🔹 결과 출력
response.pretty_print()

# RAG 시스템 접목. 

In [ ]:
import pandas as pd
# CSV 파일을 읽을 때 원래 인코딩을 지정 (예: 'cp949', 'ISO-8859-1')
df = pd.read_csv("ocr_whole.csv", encoding="cp949")

# 새로운 인코딩으로 저장 (예: 'utf-8')
df.to_csv("output.csv", encoding='cp949', index=False)
df.to_csv('ocr_final.csv', sep=',', na_rep='NaN')

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import CSVLoader, Docx2txtLoader
from langchain_openai import OpenAIEmbeddings
# from dotenv import load_dotenv

# load_dotenv()

In [ ]:
# vector store 생성. 

# Data Load 데이터 호출
loader = CSVLoader(file_path='ocr_final.csv', encoding="utf-8")
data = loader.load()


# Split 데이터 분할.
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_data = splitter.split_documents(data)
print(split_data[0])

# Embedding 임베딩
embeddings = OpenAIEmbeddings()

# Vector Store 데이터 저장. 
vector_store = Chroma.from_documents(
    documents=split_data,
    embedding=embeddings,
    persist_directory="my_vector_store",
)

page_content=': 0
작물: 가지
산도: 6.0~6.5
전기전도도: ≤2
질산태질소: 70~200
유기물: 20~30
유효인산: 400~500
칼륨: 0.70~0.80
칼슘: 5.0~6.0
마그네슘: 1.5~2.0
양이온교환용량: 10~15
붕소: NaN' metadata={'source': 'ocr_final.csv', 'row': 0}


In [ ]:
import xml.etree.ElementTree as ET
import xmltodict
import json
import requests
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

serviceKey="q+kAKJCJgJXlNlBFxk5LHCmDivqtHEVdmd3vh4cftkCafbEmv4agKxoZemYjbqE9Gxjy0lRCVmbcG3ZtR4K2Tw=="
url = 'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExam'
params ={'serviceKey' : serviceKey, 'PNU_Code' : '5115034022100750001' }

response = requests.get(url, params=params)
response_context = response.text
#### parsing to json
response_json = xmltodict.parse(response_context)["response"]
input_data = response_json["body"]["items"]["item"]

In [8]:
# 🔹 RAG를 위한 vector store 설정
embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    persist_directory="my_vector_store",
    embedding_function=embeddings,
)
retriever = vector_store.as_retriever()

# 🔹 OpenAI 모델 설정
model = ChatOpenAI(model="gpt-4o-mini")

# 🔹 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    아래의 토양 환경 정보를 기준으로 참고문서에서 키울 수 있는 가장 적합한 작물을 추천해 주세요.
    
    🌱 **사용자 입력 (토양 정보)**:
    {input_data}
    
    📄 **참고 문서 (작물별 적정 환경)**:
    {context}
    
    비교 후 가장 적합한 작물을 추천하고, 그 이유도 설명해 주세요.
    """,
    input_variables=["input_data", "context"]
)

# 벡터 DB에서 관련 문서 검색
query = "\n".join([f"{key}: {value}" for key, value in input_data.items()])
docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in docs])

# 🔹 체인 생성 및 실행
chain = prompt | model
response = chain.invoke({"input_data": input_data, "context": context})

# 🔹 결과 출력
response.pretty_print()

C:\Users\shinc\AppData\Local\Temp\ipykernel_18104\645044976.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o-mini")


================================== Ai Message ==================================

주어진 토양 환경 정보와 참고 문서에 나와 있는 각 작물의 적정 환경을 비교해보면 다음과 같습니다.

### 사용자의 토양 정보
- 산도 (pH): 6.8
- 유기물 (OM): 13.0%
- 칼륨 (K): 0.174
- 칼슘 (Ca): 3.5
- 기타 정보는 NaN으로 제공됨.

### 작물별 적정 환경
1. **포도**
   - 산도: 6.0~6.5
   - 유기물: 25~35%
   - 칼륨: 0.30~0.60
   - 칼슘: 5.0~6.0

2. **맥문동**
   - 산도: 6.0~6.5
   - 유기물: 25~35%
   - 칼륨: 0.45~0.55
   - 칼슘: 5.0~6.0

3. **꽃도라지**
   - 산도: 6.0~7.0
   - 유기물: 25~35%
   - 칼륨: 0.74~0.92
   - 칼슘: 5.0~7.0

4. **연꽃(연근)**
   - 산도: 6.0~6.5
   - 유기물: 30~50%
   - 칼륨: 0.60~0.70
   - 칼슘: 5.0~6.0

### 비교 결과
- **산도**: 사용자의 토양 pH 6.8은 꽃도라지(최대 7.0)만 적합합니다. 나머지 작물들은 산도가 6.5를 초과할 수 없습니다.
- **유기물**: 사용자의 유기물 13.0%는 모든 작물의 적정 범위(25~35%)에 미치지 못합니다.
- **칼륨**: 사용자의 칼륨 수치(0.174)는 포도와 맥문동의 적정 범위(0.30~0.60)에 미치지 못하며, 꽃도라지와 연꽃의 적정 범위(0.60~0.92)에도 부족합니다.
- **칼슘**: 사용자의 칼슘 수치(3.5)는 모든 작물의 적정 범위(5.0 이상)에 미치지 못합니다.

### 추천 작물
**꽃도라지**를 추천합니다. 

### 이유
- **산도**: 꽃도라지는 최대 pH 7.0까지 적합하므로, 사용자의 pH 6.8에 맞습니다.
- **유기물**: 유기물 수치가 

In [9]:
query = "\n".join([f"{key}: {value}" for key, value in input_data.items()])
query

'PNU_Code: 5115034022100750001\nAny_Year: 2024\nExam_Day: 20240325\nExam_Type: 1\nPNU_Nm: 강원특별자치도 강릉시 강동면 모전리 75-1\nACID: 6.800\nVLDPHA: 74.700\nVLDSIA: 77.000\nOM: 13.000\nPOSIFERT_MG: 1.100\nPOSIFERT_K: 0.174\nPOSIFERT_CA: 3.500\nSELC: 0.850'